## Install

``` shell
conda create python=2.7 h5py jupyter -p ~/space/envs/peaknet
conda install pytorch=0.1.12 torchvision cuda80 -c pytorch
conda install tensorboardX
```


### Import packages

In [ ]:
%load_ext autoreload
%autoreload 2
import torch
# import torchvision
import numpy as np
import time
from peaknet import Peaknet
from peaknet_utils import *

/reg/neh/home/liponan/space/envs/peaknet/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Load data from CXI

### Init. network

In [1]:
net = Peaknet()
net.loadCfg( "../pytorch-yolo2/cfg/newpeaksv10-asic.cfg" )
# net.init_model()
net.loadWeights( "../pytorch-yolo2/cfg/newpeaksv10-asic.cfg", 
                "../darknet/backup/newpeaksv10_10000.weights")
net.model.cuda()

NameError: name 'Peaknet' is not defined

## Train

### Dataset

In [ ]:
filename = "/reg/neh/home/liponan/data/cxic0415/r0091/cxic0415_0091.cxi.backup"
t0 = time.time()
train_imgs, train_labels, dev_imgs, dev_labels = build_dataset( filename, dev_size = 100, total_size=-1 )
t1 = time.time()
print("took " + str(t1-t0) + " s to build the datasets")
print("train_imgs", train_imgs.shape)
print("train_labels", len(train_labels), len(train_labels[0]))
print("dev_imgs", dev_imgs.shape)
print("dev_labels", len(dev_labels), len(dev_labels[0]))

### Confg

In [ ]:
project_name = "runs/adagrad_lr_001_pretrained_40000_nooptim"
n_ep = 1
init_lr = 0.01
lr_policy = 1
macro_batch_size = 4
algo = "adagrad"

params = {"n_ep":n_ep, "lr": init_lr, "batch_size":macro_batch_size, "lr_policy": lr_policy, 
          "train_size": len(train_labels), "optim": algo}

net = Peaknet()
net.loadCfg( "../pytorch-yolo2/cfg/newpeaksv10-asic.cfg" )
net.loadWeights( "../pytorch-yolo2/cfg/newpeaksv10-asic.cfg", 
                "../darknet/backup/newpeaksv10_40000.weights")
# net.init_model()
net.model.cuda()

In [ ]:
n_iters = int( np.ceil( len(train_labels) / float(macro_batch_size) ) )
net.set_writer(project_name=project_name, parameters=params)
my_lr = init_lr

for ep in range(1, n_ep+1):
    print("============= EPOCH {} ==========".format(ep))
    if ep % lr_policy == 0:
        my_lr /= 2
    for i in range(n_iters):
        if i == (n_iters-1):
            batch_imgs = train_imgs[(i*macro_batch_size):,:,:,:]
            batch_labels = train_labels[(i*macro_batch_size):]
        else:
            batch_imgs = train_imgs[i*macro_batch_size:(i+1)*macro_batch_size,:,:,:]
            batch_labels = train_labels[i*macro_batch_size:(i+1)*macro_batch_size]
        net.train( batch_imgs, batch_labels, mini_batch_size=32 )
        #net.set_optimizer(adagrad=(algo=="adagrad"), lr=my_lr )
        #net.optimize()

## Predict

In [ ]:
filename = "/reg/neh/home/liponan/data/cxic0415/r0091/cxic0415_0091.cxi.backup"
idx=468
imgs, labels = imgs, labels = load_from_cxi( filename, idx )
print("imgs", imgs.shape)
print("labels", len(labels), len(labels[0]))

In [ ]:
results = net.predict( imgs, batch_size=32, conf_thresh=0.10, use_cuda=True )


In [ ]:
print((results[14]))

In [ ]:
visualize(imgs, labels, results, plot_label=True, plot_box=True, indexes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])

In [ ]:
a, b, c, d = build_dataset( filename, dev_size = 1, total_size=10 )